In [10]:
#Step 1-Install libraries
!pip install earthengine-api geemap
!pip install -U geemap
!pip install geopandas
!pip install pycrs
import ee
import geemap
from google.colab import drive

#Step 2-Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='myprojectstream78691')

#Step 3- Mount Google Drive
drive.mount('/content/drive')

# Step4- Define AOI Coordinates
aoi_coordinates = [
    [-88.7084, 35.4405],
    [-89.2687, 35.4405],
    [-89.2687, 35.0328],
    [-88.7084, 35.0328]
]

# Step 5- Create geometry from the AOI
aoi = ee.Geometry.Polygon(aoi_coordinates)

#Step 6- Create 50m buffer around aoi
buf_aoi = aoi.buffer(50)

# Step7- Filter Sentinel-2 Harmonized Collection by date and cloud cover
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(buf_aoi)
    .filterDate("2022-12-01", "2023-02-28")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
    .sort("CLOUDY_PIXEL_PERCENTAGE")
)

#Step 8- Create a mosaic of the images to cover the entire buffered aoi and calculate ndvi
#should i use .mosaic or .median??????
mos_sen = collection.mosaic().clip(buf_aoi)

#Step 9- Calculate ndvi for mosaiced aoi and renaming as NDVI
ndvi_aoi = mos_sen.normalizedDifference(["B8", "B4"]).rename("NDVI")

# Step10- Calculate mean ndvi over the buffered aoi
mean_ndvi = ndvi_aoi.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=buf_aoi,
    scale=10,
    maxPixels=1e9
).get('NDVI').getInfo()

#Step 11- Print mean ndvi value
print("Mean ndvi for the buffered aoi:", mean_ndvi)

#Step 12- Create a constant image of mean ndvi
cons_mean_ndvi= ee.Image.constant(mean_ndvi).clip(buf_aoi)

# Step13- Define visualization parameters
rgb_vis_params = {
    "min": 0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],  # RGB bands for true color
}

mean_ndvi_vis_params = {
    "min": -1,
    "max": 1,
    "palette": ["blue", "white", "green"]
}

# Step14- Display the Mosaic Image and Mean NDVI Layer on the Map
Map = geemap.Map(center=[35.2, -88.9], zoom=9)
Map.addLayer(mos_sen, rgb_vis_params, "Mosaic Image (Sentinel-2)")
Map.addLayer(buf_aoi, {"color": "yellow"}, "Buffered AOI Boundary")
Map.addLayer(cons_mean_ndvi, mean_ndvi_vis_params, "Mean NDVI Layer")

#Step 15- Display the map
Map

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mean ndvi for the buffered aoi: 0.4399704771302369


Map(center=[35.2, -88.9], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [17]:
#Bond Creek

#Step 1-Install libraries
!pip install earthengine-api geemap
!pip install -U geemap
!pip install geopandas
!pip install pycrs
import ee
import geemap
from google.colab import drive

#Step 2-Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='myprojectstream78691')

#Step 3- Mount Google Drive
drive.mount('/content/drive')

#Step4-Define aoi geometry from GEE asset
       #Bond Creek tributaries polyline shapefile
aoi_bond = ee.FeatureCollection("projects/myprojectstream78691/assets/bond_stpl_polyline").geometry()

#Step 5- Create 50m buffer around aoi
buf_aoi_bond = aoi_bond.buffer(50)

#Step6- Filter Sentinel-2 Harmonized Collection by date and cloud cover
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(buf_aoi_bond)
    .filterDate("2022-12-01", "2023-02-28")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
    .sort("CLOUDY_PIXEL_PERCENTAGE")
)

#Step 7- Create a mosaic of the images to cover the entire buffered aoi
#.mosaic or .median??????????????
mos_bond = collection.mosaic().clip(buf_aoi_bond)
#median_bond=collection.median().clip(buf_aoi_bond)

#Step 8- Calculate ndvi for mosaiced aoi and renaming as NDVI_Bond
ndvi_bond = mos_bond.normalizedDifference(["B8", "B4"]).rename("NDVI_Bond")

# Step9- Calculate mean ndvi over the buffered aoi
mean_ndvi_bond = ndvi_bond.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=buf_aoi_bond,
    scale=10,
    maxPixels=1e9
).get('NDVI_Bond').getInfo()

#Step 10- Print mean ndvi value
print("Mean ndvi for the buffered aoi:", mean_ndvi_bond)

#Step 11- Create a constant image of mean ndvi
cons_mean_ndvi_bond= ee.Image.constant(mean_ndvi_bond).clip(buf_aoi_bond)

#Step12-Define visualization parameters(True color)
rgb_vis_params = {
    "min": 0,
    "max": 3000,  ##found other codes use it. Do not know why 3000?
    "bands": ["B4", "B3", "B2"],
}

mean_ndvi_vis_params = {
    "min": -1,
    "max": 1,
    "palette": ["blue", "white", "green"]
}

# Step13- Display mosaiced image,mean ndvi and buffered aoi
Map = geemap.Map(center=[35.2, -88.9], zoom=9)
Map.addLayer(aoi_bond, {"color": "blue"}, "Bond Creek AOI")
Map.addLayer(mos_bond, rgb_vis_params, "Mosaic Image (Sentinel-2)")
Map.addLayer(buf_aoi_bond, {"color": "yellow"}, "Buffered AOI Boundary")
Map.addLayer(cons_mean_ndvi_bond, mean_ndvi_vis_params, "Mean NDVI Layer")



# Step 14- Display the map
Map


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mean ndvi for the buffered aoi: 0.32953696787773146


Map(center=[35.2, -88.9], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…